In [1]:
# en_core_web_sm(12 mb)
# en_core_web_md(43 mb)
# en_core_web_lg(741 mb)
# en_core_web_trf(438 mb)

In [11]:
import spacy 

nlp = spacy.load("en_core_web_sm")
nlp.to_disk("defaultModel")
#check for ner pipleine is available or not
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [3]:
#1. docx to text
import docx2txt

# extract text
# file_name="CV Wyc.docx"
file_name="HarishBharatMali[7_0]-converted.docx"
text = docx2txt.process(f"./ResumeData/{file_name}")
# text
with open(f"./txtData/{file_name}.txt","w") as f:
    f.write(text)
    

In [ ]:
#### With default model result

In [58]:
# sentence = "Daniil Medvedev and Novak Djokovic have built an intriguing rivalry since the Australian Open decider, which the Serb won comprehensively."
doc = nlp(text)

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [14]:
#2. Annotate the txt file

In [15]:
#3. Convet .jsonl to .spacy

In [4]:
#select file
annotation_file_name="./AnnotationJSONL/combined_root.jsonl"

In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import logging
import json

#filtter data to convert in spacy format
def fillterDoccanoData(doccano_JSONL_FilePath):
    try:
        training_data = []
        lines=[]
        with open(doccano_JSONL_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['data']
            entities = data['label']
            if len(entities)>0:
                training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + doccano_JSONL_FilePath + "\n" + "error = " + str(e))
        return None

In [6]:
#read Doccano Annotation file .jsonl
TRAIN_DATA=fillterDoccanoData(annotation_file_name)



In [9]:
# with open ("spacyforamt.json", "w") as f:
#     f.write(TRAIN_DATA)
    
# Serializing json 
json_object = json.dumps(TRAIN_DATA)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
import json
json.dump()

In [7]:
TRAIN_DATA

[('CURRICULUM VITAE SUMMARY\n\nWycliff Kabogo Atunga\t                                           +256 756104040\t\t              kawycliff3@gmail.com\n\nOBJECTIVE:\n\n\n\nI seek to work in an organization where i will contribute to its objectives, support its clients and learn continuously.\n\n\n\nA highly organized Supply Chain Management Executive with over 10 years of experience spearheading International Procurement Sourcing, Formulating of Multi-clientele Acquisition Plans, Negotiating Global Contracts, Scheduling Multi-national projects Supply Chain activities , Integrating and consolidating acquisition processes through ERP system, Material Resource Planning (MRP), Establishing new procurement partnerships, designing and implementing acquisition processes such as local bid evaluation processing, enhancing supplier relationship, bench-marking, assets management, Shipping and clearing, fleet management and Sustainable purchasing policies for goods, services and works projects, inv

In [39]:
 #root.jsonl is annotation file name file name 

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    try:
        doc.ents = ents # label the text with the ents
        db.add(doc)
    except:
        print(text, annot)
db.to_disk("./SpaCyTrainData/train.spacy") # save the docbin object

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 63.26it/s]

Skipping entity


In [ ]:
#Load Config file

In [20]:
!python -m spacy init config --lang en --pipeline ner config.cfg --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [63]:
!python -m spacy train config.cfg --output ./training/ --paths.train ./SpaCyTrainData/train.spacy --paths.dev ./SpaCyTrainData/dev.spacy --training.eval_frequency 10 --training.max_steps 200 --gpu-id -0


#Explain:
#<python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy>
#Dev --> Validation Data 
#train --> Training Data 

ℹ Saving to output directory: training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-04-04 10:07:04,872] [INFO] Set up nlp object from config
[2022-04-04 10:07:04,879] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-04 10:07:04,882] [INFO] Created vocabulary
[2022-04-04 10:07:04,882] [INFO] Finished initializing nlp object
[2022-04-04 10:07:05,464] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    717.48    0.00    0.00    0.00    0.00
  5      10         29.94   9497.06    0.00    0.00    0.00    0.00
 10      20        236.97   1503.03    0.00    0.00    0.00    0.00
 15      30        228.85    939.15   

In [10]:
# config.cfg

In [42]:
#load trained custom model


In [43]:
custNlp = spacy.load(r"./training/model-best") #load the best model


In [44]:
#Test on New data

In [45]:
import docx2txt
Cv_PathToTest="./ResumeData/HarishBharatMali[7_0]-converted.docx"
cv_text = docx2txt.process(Cv_PathToTest)

In [46]:
doc = custNlp(cv_text) # input sample tex

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter
[(X, X.ent_type_) for X in doc if X.ent_type_]

[(Harish, 'Name'),
 (Bharat, 'Name'),
 (Mali, 'Name'),
 (harish15mali@gmail.com, 'Email'),
 (+91, 'PhoneNumber'),
 (6.8, 'TotalExperience'),
 (years, 'TotalExperience'),
 (of, 'TotalExperience'),
 (experience, 'TotalExperience'),
 (Oracle, 'Skills'),
 (,, 'Skills'),
 (Cassandra, 'Skills'),
 (Database, 'Skills'),
 (Process, 'Skills'),
 (Associate, 'Skills'),
 (and, 'Skills'),
 (Database, 'Skills'),
 (Administrator, 'Skills'),
 (., 'Skills'),
 (Tata, 'CompanyName'),
 (Consultancy, 'CompanyName'),
 (Services, 'CompanyName'),
 (Limited, 'CompanyName'),
 (Pune, 'Location'),
 (Tata, 'CompanyName'),
 (Consultancy, 'CompanyName'),
 (Services, 'CompanyName'),
 (Limited, 'CompanyName'),
 (FUTURZ, 'CompanyName'),
 (Database, 'Skills'),
 (Administrator, 'Skills'),
 (Oracle, 'Skills'),
 (Cassandra, 'Skills'),
 (Linux, 'Skills'),
 (Unix, 'Skills'),
 (Oracle, 'Skills'),
 (12c, 'Skills'),
 (Oracle, 'Skills'),
 (19c, 'Skills'),
 (Cassandra, 'Skills'),
 (ServiceNow, 'Skills'),
 (Kibana, 'Skills'),
 (Tal

In [48]:
## other

import docx2txt
Cv_PathToTest="./ResumeData/CV Wyc.docx"
cv_text_n = docx2txt.process(Cv_PathToTest)

###################################################
doc = custNlp(cv_text_n) # input sample tex

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter
[(X, X.ent_type_) for X in doc if X.ent_type_]

[(Wycliff, 'Name'),
 (Kabogo, 'Name'),
 (Atunga, 'Name'),
 (+256, 'PhoneNumber'),
 (756104040, 'PhoneNumber'),
 (kawycliff3@gmail.com, 'Email'),
 (Supply, 'Designation'),
 (Chain, 'Designation'),
 (Management, 'Designation'),
 (Executive, 'Designation'),
 (10, 'TotalExperience'),
 (years, 'TotalExperience'),
 (Material, 'Skills'),
 (Resource, 'Skills'),
 (Planning, 'Skills'),
 ((, 'Skills'),
 (MRP, 'Skills'),
 (), 'Skills'),
 (and, 'Skills'),
 (clearing, 'Skills'),
 (,, 'Skills'),
 (fleet, 'Skills'),
 (management, 'Skills'),
 (Certified, 'Certification'),
 (International, 'Certification'),
 (Supply, 'Certification'),
 (Chain, 'Certification'),
 (Management, 'Certification'),
 (Professional, 'Certification'),
 ((, 'Certification'),
 (CISCP, 'Certification'),
 (), 'Certification'),
 (Bachelor, 'Degree'),
 (of, 'Degree'),
 (Business, 'Degree'),
 (Administration, 'Degree'),
 (MUBS, 'Degree'),
 ((, 'Degree'),
 (Makerere, 'Degree'),
 (University, 'Degree'),
 (Business, 'Degree'),
 (School, '

In [ ]:
######################################################
######################################################
# Default

In [49]:
import spacy 

# nlp = spacy.load("en_core_web_sm")
#check for ner pipleine is available or not
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [59]:
#read CV
import docx2txt
cv_text = docx2txt.process("./ResumeData/VinothRajendran[6_0]-converted.docx")
# print(type(cv_text))

In [61]:
doc = custNlp(cv_text)

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

## Custom NER:
SpaCy pre-trained models detect and categorize the text chunks into 18 types of entities.

Steps to build the custom NER model for detecting the job role in job postings in spaCy 3.0:

1. Annotate the data to train the model.
2. Convert the annotated data into the spaCy bin object.
3. Generate the config file from the spaCy website.
4. Train the model in the command line.
5. Load and test the saved model.